## 机器学习策略——基于逻辑回归的股市趋势预测(及SVM策略)

In [2]:
import pandas as pd 
import numpy as np
import copy
import datetime
import os
import glob
import copy
import datetime
from struct import *
from pandas.tseries.offsets import DateOffset,Day

### 数据源处理，读取通达信数据，并保存为CSV格式
save_alldata_from_tdx（） 读取通达信数据
getstockdata（）获取csv文件

In [3]:
#定义文件读取路径
def save_5min_tdx(f_add='D:/Program Files (x86)/tdx/vipdoc/sz/fzline/',s_add='D:/tushare/data/5min/' ):
    '''
    读取通达信软件下的5分钟数据，并转换为CSV格式，存入'D:/tushare/data/5min/'
    '''
    
    os.chdir(f_add)
    files_list = glob.glob('*.lc5')
    print(len(files_list))
    address = os.getcwd()#返回当前目录

    for j in range(len(files_list)):
        if (files_list[j][2]=='1'):
            print('delet '+files_list[j]+'!')
            continue;
        try:
            ofile=open(files_list[j],'rb')
            buf=ofile.read()
            ofile.close()
            
            num=len(buf)
            no=int(num/32)
            b=0
            e=32
            dl = []
            for i in range(no):
                a=unpack('hhfffffii',buf[b:e])
                dl.append([str(int(a[0]/2048)+2004)+'-'+str(int(a[0]%2048/100)).zfill(2)+'-'+str(a[0]%2048%100).zfill(2),str(int(a[1]/60)).zfill(2)+':'+str(a[1]%60).zfill(2)+':00',a[2],a[3],a[4],a[5],a[6],a[7]])
                b=b+32
                e=e+32
            df = pd.DataFrame(dl, columns=['date','time','open','high','low','close','amount','vol'])
            df.insert(2,'code',files_list[j][0:8])#将代码加入
                        
            df.to_csv(s_add+files_list[j][0:8]+'.csv',sep=',',encoding='utf8')#将文件保存为csv格式
            print(files_list[j][0:8] +' has saved on '+s_add )
        except UnicodeDecodeError:
            print(files_list[i] +'gb2312 codec can not decode')
        except:
            print(files_list[i] +" has Unexpected error:")      

In [22]:
#定义文件读取路径
def save_d_tdx(f_add='D:/Program Files (x86)/tdx/vipdoc/sz/lday/',s_add='D:/tushare/data/day/' ):
    '''
    读取通达信软件下的日线数据，并转换为CSV格式，存入'D:/tushare/data/day/'
    '''
    os.chdir(f_add)
    files_list = glob.glob('*.day')
    print(len(files_list))
    address = os.getcwd()#返回当前目录

    for j in range(len(files_list)):
        if (files_list[j][2]=='1'):
            print('delet '+files_list[j]+'!')
            continue;
        
        try:
            dayfile = open(files_list[j], 'rb')
            buffer = dayfile.read()
            dayfile.close()
            #读取的数据长度
            buflengh = len(buffer) #32的整数倍
            #除32获利文件中存储了多少天的信息
            datenum = int(buflengh / 32)
            #解析每一天的数据的内容,每一天列表的起始位置和终止位置
            list_begin = 0
            list_end = 32
            #创建pandas股票数据表
            #seriesdata = pd.Series() #创建列表
            #stockdata = pd.DataFrame(index =range(0,datenum),columns =['date','open','high','low','close','amount','volum'])
            #将个股数据转换为pandas的dataFrame格式数据
            dl =[]
            for i in range(datenum):
                #解析32个字节一天数据
                a = unpack('IIIIIfII', buffer[list_begin:list_end])
                date1 = str(a[0])+' 14:55:00'
                date  = pd.to_datetime(date1)
                open1 = a[1] / 100.0
                high  = a[2] / 100.0
                low   = a[3] / 100.0
                close = a[4] / 100.0
                amount = a[5] / 10.0
                vol = a[6]
                unused = a[7]
                dl.append([date,open1,high,low,close,amount,vol])
                #加32指向下一天的数据列表
                list_begin = list_begin + 32
                list_end = list_end + 32

            df = pd.DataFrame(dl, columns=['date','open','high','low','close','amount','vol'])

            #生成对应的文件
            savefilename = files_list[j][0:8]
            df.insert(1,'code',savefilename)#将代码加入
            df.to_csv(s_add + os.sep + savefilename +' .csv', encoding='utf8')
            print (savefilename + 'has been saved in'+ s_add+'!' )
        
        except UnicodeDecodeError:
            print(savefilename +'gb2312 codec can not decode')
        except:
            print(savefilename +" has Unexpected error:") 


In [23]:
save_d_tdx()

2875
sz000001has been saved inD:/tushare/data/day/!
sz000002has been saved inD:/tushare/data/day/!
sz000004has been saved inD:/tushare/data/day/!
sz000005has been saved inD:/tushare/data/day/!
sz000006has been saved inD:/tushare/data/day/!
sz000007has been saved inD:/tushare/data/day/!
sz000008has been saved inD:/tushare/data/day/!
sz000009has been saved inD:/tushare/data/day/!
sz000010has been saved inD:/tushare/data/day/!
sz000011has been saved inD:/tushare/data/day/!
sz000012has been saved inD:/tushare/data/day/!
sz000014has been saved inD:/tushare/data/day/!
sz000016has been saved inD:/tushare/data/day/!
sz000017has been saved inD:/tushare/data/day/!
sz000018has been saved inD:/tushare/data/day/!
sz000019has been saved inD:/tushare/data/day/!
sz000020has been saved inD:/tushare/data/day/!
sz000021has been saved inD:/tushare/data/day/!
sz000022has been saved inD:/tushare/data/day/!
sz000023has been saved inD:/tushare/data/day/!
sz000025has been saved inD:/tushare/data/day/!
sz000026

sz000596has been saved inD:/tushare/data/day/!
sz000597has been saved inD:/tushare/data/day/!
sz000598has been saved inD:/tushare/data/day/!
sz000599has been saved inD:/tushare/data/day/!
sz000600has been saved inD:/tushare/data/day/!
sz000601has been saved inD:/tushare/data/day/!
sz000603has been saved inD:/tushare/data/day/!
sz000605has been saved inD:/tushare/data/day/!
sz000606has been saved inD:/tushare/data/day/!
sz000607has been saved inD:/tushare/data/day/!
sz000608has been saved inD:/tushare/data/day/!
sz000609has been saved inD:/tushare/data/day/!
sz000610has been saved inD:/tushare/data/day/!
sz000611has been saved inD:/tushare/data/day/!
sz000612has been saved inD:/tushare/data/day/!
sz000613has been saved inD:/tushare/data/day/!
sz000615has been saved inD:/tushare/data/day/!
sz000616has been saved inD:/tushare/data/day/!
sz000617has been saved inD:/tushare/data/day/!
sz000619has been saved inD:/tushare/data/day/!
sz000620has been saved inD:/tushare/data/day/!
sz000622has b

sz000848has been saved inD:/tushare/data/day/!
sz000850has been saved inD:/tushare/data/day/!
sz000851has been saved inD:/tushare/data/day/!
sz000852has been saved inD:/tushare/data/day/!
sz000856has been saved inD:/tushare/data/day/!
sz000858has been saved inD:/tushare/data/day/!
sz000859has been saved inD:/tushare/data/day/!
sz000860has been saved inD:/tushare/data/day/!
sz000861has been saved inD:/tushare/data/day/!
sz000862has been saved inD:/tushare/data/day/!
sz000863has been saved inD:/tushare/data/day/!
sz000868has been saved inD:/tushare/data/day/!
sz000869has been saved inD:/tushare/data/day/!
sz000875has been saved inD:/tushare/data/day/!
sz000876has been saved inD:/tushare/data/day/!
sz000877has been saved inD:/tushare/data/day/!
sz000878has been saved inD:/tushare/data/day/!
sz000880has been saved inD:/tushare/data/day/!
sz000881has been saved inD:/tushare/data/day/!
sz000882has been saved inD:/tushare/data/day/!
sz000883has been saved inD:/tushare/data/day/!
sz000885has b

sz002062has been saved inD:/tushare/data/day/!
sz002063has been saved inD:/tushare/data/day/!
sz002064has been saved inD:/tushare/data/day/!
sz002065has been saved inD:/tushare/data/day/!
sz002066has been saved inD:/tushare/data/day/!
sz002067has been saved inD:/tushare/data/day/!
sz002068has been saved inD:/tushare/data/day/!
sz002069has been saved inD:/tushare/data/day/!
sz002070has been saved inD:/tushare/data/day/!
sz002071has been saved inD:/tushare/data/day/!
sz002072has been saved inD:/tushare/data/day/!
sz002073has been saved inD:/tushare/data/day/!
sz002074has been saved inD:/tushare/data/day/!
sz002075has been saved inD:/tushare/data/day/!
sz002076has been saved inD:/tushare/data/day/!
sz002077has been saved inD:/tushare/data/day/!
sz002078has been saved inD:/tushare/data/day/!
sz002079has been saved inD:/tushare/data/day/!
sz002080has been saved inD:/tushare/data/day/!
sz002081has been saved inD:/tushare/data/day/!
sz002082has been saved inD:/tushare/data/day/!
sz002083has b

sz002237has been saved inD:/tushare/data/day/!
sz002238has been saved inD:/tushare/data/day/!
sz002239has been saved inD:/tushare/data/day/!
sz002240has been saved inD:/tushare/data/day/!
sz002241has been saved inD:/tushare/data/day/!
sz002242has been saved inD:/tushare/data/day/!
sz002243has been saved inD:/tushare/data/day/!
sz002244has been saved inD:/tushare/data/day/!
sz002245has been saved inD:/tushare/data/day/!
sz002246has been saved inD:/tushare/data/day/!
sz002247has been saved inD:/tushare/data/day/!
sz002248has been saved inD:/tushare/data/day/!
sz002249has been saved inD:/tushare/data/day/!
sz002250has been saved inD:/tushare/data/day/!
sz002251has been saved inD:/tushare/data/day/!
sz002252has been saved inD:/tushare/data/day/!
sz002253has been saved inD:/tushare/data/day/!
sz002254has been saved inD:/tushare/data/day/!
sz002255has been saved inD:/tushare/data/day/!
sz002256has been saved inD:/tushare/data/day/!
sz002258has been saved inD:/tushare/data/day/!
sz002259has b

...

sz200011has been saved inD:/tushare/data/day/!
sz200012has been saved inD:/tushare/data/day/!
sz200016has been saved inD:/tushare/data/day/!
sz200017has been saved inD:/tushare/data/day/!
sz200018has been saved inD:/tushare/data/day/!
sz200019has been saved inD:/tushare/data/day/!
sz200020has been saved inD:/tushare/data/day/!
sz200022has been saved inD:/tushare/data/day/!
sz200025has been saved inD:/tushare/data/day/!
sz200026has been saved inD:/tushare/data/day/!
sz200028has been saved inD:/tushare/data/day/!
sz200029has been saved inD:/tushare/data/day/!
sz200030has been saved inD:/tushare/data/day/!
sz200037has been saved inD:/tushare/data/day/!
sz200045has been saved inD:/tushare/data/day/!
sz200054has been saved inD:/tushare/data/day/!
sz200055has been saved inD:/tushare/data/day/!
sz200056has been saved inD:/tushare/data/day/!
sz200058has been saved inD:/tushare/data/day/!
sz200152has been saved inD:/tushare/data/day/!
sz200160has been saved inD:/tushare/data/day/!
sz200168has b

sz300128has been saved inD:/tushare/data/day/!
sz300129has been saved inD:/tushare/data/day/!
sz300130has been saved inD:/tushare/data/day/!
sz300131has been saved inD:/tushare/data/day/!
sz300132has been saved inD:/tushare/data/day/!
sz300133has been saved inD:/tushare/data/day/!
sz300134has been saved inD:/tushare/data/day/!
sz300135has been saved inD:/tushare/data/day/!
sz300136has been saved inD:/tushare/data/day/!
sz300137has been saved inD:/tushare/data/day/!
sz300138has been saved inD:/tushare/data/day/!
sz300139has been saved inD:/tushare/data/day/!
sz300140has been saved inD:/tushare/data/day/!
sz300141has been saved inD:/tushare/data/day/!
sz300142has been saved inD:/tushare/data/day/!
sz300143has been saved inD:/tushare/data/day/!
sz300144has been saved inD:/tushare/data/day/!
sz300145has been saved inD:/tushare/data/day/!
sz300146has been saved inD:/tushare/data/day/!
sz300147has been saved inD:/tushare/data/day/!
sz300148has been saved inD:/tushare/data/day/!
sz300149has b

sz300304has been saved inD:/tushare/data/day/!
sz300305has been saved inD:/tushare/data/day/!
sz300306has been saved inD:/tushare/data/day/!
sz300307has been saved inD:/tushare/data/day/!
sz300308has been saved inD:/tushare/data/day/!
sz300309has been saved inD:/tushare/data/day/!
sz300310has been saved inD:/tushare/data/day/!
sz300311has been saved inD:/tushare/data/day/!
sz300312has been saved inD:/tushare/data/day/!
sz300313has been saved inD:/tushare/data/day/!
sz300314has been saved inD:/tushare/data/day/!
sz300315has been saved inD:/tushare/data/day/!
sz300316has been saved inD:/tushare/data/day/!
sz300317has been saved inD:/tushare/data/day/!
sz300318has been saved inD:/tushare/data/day/!
sz300319has been saved inD:/tushare/data/day/!
sz300320has been saved inD:/tushare/data/day/!
sz300321has been saved inD:/tushare/data/day/!
sz300322has been saved inD:/tushare/data/day/!
sz300323has been saved inD:/tushare/data/day/!
sz300324has been saved inD:/tushare/data/day/!
sz300325has b

sz300482has been saved inD:/tushare/data/day/!
sz300483has been saved inD:/tushare/data/day/!
sz300484has been saved inD:/tushare/data/day/!
sz300485has been saved inD:/tushare/data/day/!
sz300486has been saved inD:/tushare/data/day/!
sz300487has been saved inD:/tushare/data/day/!
sz300488has been saved inD:/tushare/data/day/!
sz300489has been saved inD:/tushare/data/day/!
sz300490has been saved inD:/tushare/data/day/!
sz300491has been saved inD:/tushare/data/day/!
sz300492has been saved inD:/tushare/data/day/!
sz300493has been saved inD:/tushare/data/day/!
sz300494has been saved inD:/tushare/data/day/!
sz300495has been saved inD:/tushare/data/day/!
sz300496has been saved inD:/tushare/data/day/!
sz300497has been saved inD:/tushare/data/day/!
sz300498has been saved inD:/tushare/data/day/!
sz300499has been saved inD:/tushare/data/day/!
sz300500has been saved inD:/tushare/data/day/!
sz300501has been saved inD:/tushare/data/day/!
sz300502has been saved inD:/tushare/data/day/!
sz300503has b

sz300666has been saved inD:/tushare/data/day/!
sz300667has been saved inD:/tushare/data/day/!
sz300668has been saved inD:/tushare/data/day/!
sz300669has been saved inD:/tushare/data/day/!
sz300670has been saved inD:/tushare/data/day/!
sz300671has been saved inD:/tushare/data/day/!
sz300672has been saved inD:/tushare/data/day/!
sz300673has been saved inD:/tushare/data/day/!
sz300674has been saved inD:/tushare/data/day/!
sz300675has been saved inD:/tushare/data/day/!
sz300676has been saved inD:/tushare/data/day/!
sz300677has been saved inD:/tushare/data/day/!
sz300678has been saved inD:/tushare/data/day/!
sz300679has been saved inD:/tushare/data/day/!
sz300680has been saved inD:/tushare/data/day/!
sz300681has been saved inD:/tushare/data/day/!
sz300682has been saved inD:/tushare/data/day/!
sz300683has been saved inD:/tushare/data/day/!
sz300684has been saved inD:/tushare/data/day/!
sz300685has been saved inD:/tushare/data/day/!
sz300686has been saved inD:/tushare/data/day/!
sz300687has b

In [24]:
save_5min_tdx()

2858
sz000001 has saved on D:/tushare/data/5min/
sz000002 has saved on D:/tushare/data/5min/
sz000004 has saved on D:/tushare/data/5min/
sz000005 has saved on D:/tushare/data/5min/
sz000006 has saved on D:/tushare/data/5min/
sz000007 has saved on D:/tushare/data/5min/
sz000008 has saved on D:/tushare/data/5min/
sz000009 has saved on D:/tushare/data/5min/
sz000010 has saved on D:/tushare/data/5min/
sz000011 has saved on D:/tushare/data/5min/
sz000012 has saved on D:/tushare/data/5min/
sz000014 has saved on D:/tushare/data/5min/
sz000016 has saved on D:/tushare/data/5min/
sz000017 has saved on D:/tushare/data/5min/
sz000018 has saved on D:/tushare/data/5min/
sz000019 has saved on D:/tushare/data/5min/
sz000020 has saved on D:/tushare/data/5min/
sz000021 has saved on D:/tushare/data/5min/
sz000023 has saved on D:/tushare/data/5min/
sz000025 has saved on D:/tushare/data/5min/
sz000026 has saved on D:/tushare/data/5min/
sz000027 has saved on D:/tushare/data/5min/
sz000028 has saved on D:/tu

sz000612 has saved on D:/tushare/data/5min/
sz000613 has saved on D:/tushare/data/5min/
sz000615 has saved on D:/tushare/data/5min/
sz000616 has saved on D:/tushare/data/5min/
sz000617 has saved on D:/tushare/data/5min/
sz000619 has saved on D:/tushare/data/5min/
sz000620 has saved on D:/tushare/data/5min/
sz000622 has saved on D:/tushare/data/5min/
sz000623 has saved on D:/tushare/data/5min/
sz000625 has saved on D:/tushare/data/5min/
sz000626 has saved on D:/tushare/data/5min/
sz000627 has saved on D:/tushare/data/5min/
sz000628 has saved on D:/tushare/data/5min/
sz000629 has saved on D:/tushare/data/5min/
sz000630 has saved on D:/tushare/data/5min/
sz000631 has saved on D:/tushare/data/5min/
sz000632 has saved on D:/tushare/data/5min/
sz000633 has saved on D:/tushare/data/5min/
sz000635 has saved on D:/tushare/data/5min/
sz000636 has saved on D:/tushare/data/5min/
sz000637 has saved on D:/tushare/data/5min/
sz000638 has saved on D:/tushare/data/5min/
sz000639 has saved on D:/tushare

sz000892 has saved on D:/tushare/data/5min/
sz000893 has saved on D:/tushare/data/5min/
sz000895 has saved on D:/tushare/data/5min/
sz000897 has saved on D:/tushare/data/5min/
sz000898 has saved on D:/tushare/data/5min/
sz000899 has saved on D:/tushare/data/5min/
sz000900 has saved on D:/tushare/data/5min/
sz000901 has saved on D:/tushare/data/5min/
sz000902 has saved on D:/tushare/data/5min/
sz000903 has saved on D:/tushare/data/5min/
sz000905 has saved on D:/tushare/data/5min/
sz000906 has saved on D:/tushare/data/5min/
sz000908 has saved on D:/tushare/data/5min/
sz000909 has saved on D:/tushare/data/5min/
sz000910 has saved on D:/tushare/data/5min/
sz000911 has saved on D:/tushare/data/5min/
sz000912 has saved on D:/tushare/data/5min/
sz000913 has saved on D:/tushare/data/5min/
sz000915 has saved on D:/tushare/data/5min/
sz000917 has saved on D:/tushare/data/5min/
sz000918 has saved on D:/tushare/data/5min/
sz000919 has saved on D:/tushare/data/5min/
sz000920 has saved on D:/tushare

sz002103 has saved on D:/tushare/data/5min/
sz002104 has saved on D:/tushare/data/5min/
sz002105 has saved on D:/tushare/data/5min/
sz002106 has saved on D:/tushare/data/5min/
sz002107 has saved on D:/tushare/data/5min/
sz002108 has saved on D:/tushare/data/5min/
sz002109 has saved on D:/tushare/data/5min/
sz002110 has saved on D:/tushare/data/5min/
sz002111 has saved on D:/tushare/data/5min/
sz002112 has saved on D:/tushare/data/5min/
sz002113 has saved on D:/tushare/data/5min/
sz002114 has saved on D:/tushare/data/5min/
sz002115 has saved on D:/tushare/data/5min/
sz002116 has saved on D:/tushare/data/5min/
sz002117 has saved on D:/tushare/data/5min/
sz002118 has saved on D:/tushare/data/5min/
sz002119 has saved on D:/tushare/data/5min/
sz002120 has saved on D:/tushare/data/5min/
sz002121 has saved on D:/tushare/data/5min/
sz002122 has saved on D:/tushare/data/5min/
sz002123 has saved on D:/tushare/data/5min/
sz002124 has saved on D:/tushare/data/5min/
sz002125 has saved on D:/tushare

sz002291 has saved on D:/tushare/data/5min/
sz002292 has saved on D:/tushare/data/5min/
sz002293 has saved on D:/tushare/data/5min/
sz002294 has saved on D:/tushare/data/5min/
sz002295 has saved on D:/tushare/data/5min/
sz002296 has saved on D:/tushare/data/5min/
sz002297 has saved on D:/tushare/data/5min/
sz002298 has saved on D:/tushare/data/5min/
sz002299 has saved on D:/tushare/data/5min/
sz002300 has saved on D:/tushare/data/5min/
sz002301 has saved on D:/tushare/data/5min/
sz002302 has saved on D:/tushare/data/5min/
sz002303 has saved on D:/tushare/data/5min/
sz002304 has saved on D:/tushare/data/5min/
sz002305 has saved on D:/tushare/data/5min/
sz002306 has saved on D:/tushare/data/5min/
sz002307 has saved on D:/tushare/data/5min/
sz002308 has saved on D:/tushare/data/5min/
sz002309 has saved on D:/tushare/data/5min/
sz002310 has saved on D:/tushare/data/5min/
sz002311 has saved on D:/tushare/data/5min/
sz002312 has saved on D:/tushare/data/5min/
sz002313 has saved on D:/tushare

...

sz200011 has saved on D:/tushare/data/5min/
sz200012 has saved on D:/tushare/data/5min/
sz200016 has saved on D:/tushare/data/5min/
sz200017 has saved on D:/tushare/data/5min/
sz200018 has saved on D:/tushare/data/5min/
sz200019 has saved on D:/tushare/data/5min/
sz200020 has saved on D:/tushare/data/5min/
sz200025 has saved on D:/tushare/data/5min/
sz200026 has saved on D:/tushare/data/5min/
sz200028 has saved on D:/tushare/data/5min/
sz200029 has saved on D:/tushare/data/5min/
sz200030 has saved on D:/tushare/data/5min/
sz200037 has saved on D:/tushare/data/5min/
sz200045 has saved on D:/tushare/data/5min/
sz200054 has saved on D:/tushare/data/5min/
sz200055 has saved on D:/tushare/data/5min/
sz200056 has saved on D:/tushare/data/5min/
sz200058 has saved on D:/tushare/data/5min/
sz200152 has saved on D:/tushare/data/5min/
sz200160 has saved on D:/tushare/data/5min/
sz200168 has saved on D:/tushare/data/5min/
sz200413 has saved on D:/tushare/data/5min/
sz200418 has saved on D:/tushare

sz300141 has saved on D:/tushare/data/5min/
sz300142 has saved on D:/tushare/data/5min/
sz300143 has saved on D:/tushare/data/5min/
sz300144 has saved on D:/tushare/data/5min/
sz300145 has saved on D:/tushare/data/5min/
sz300146 has saved on D:/tushare/data/5min/
sz300147 has saved on D:/tushare/data/5min/
sz300148 has saved on D:/tushare/data/5min/
sz300149 has saved on D:/tushare/data/5min/
sz300150 has saved on D:/tushare/data/5min/
sz300151 has saved on D:/tushare/data/5min/
sz300152 has saved on D:/tushare/data/5min/
sz300153 has saved on D:/tushare/data/5min/
sz300154 has saved on D:/tushare/data/5min/
sz300155 has saved on D:/tushare/data/5min/
sz300156 has saved on D:/tushare/data/5min/
sz300157 has saved on D:/tushare/data/5min/
sz300158 has saved on D:/tushare/data/5min/
sz300159 has saved on D:/tushare/data/5min/
sz300160 has saved on D:/tushare/data/5min/
sz300161 has saved on D:/tushare/data/5min/
sz300162 has saved on D:/tushare/data/5min/
sz300163 has saved on D:/tushare

sz300329 has saved on D:/tushare/data/5min/
sz300330 has saved on D:/tushare/data/5min/
sz300331 has saved on D:/tushare/data/5min/
sz300332 has saved on D:/tushare/data/5min/
sz300333 has saved on D:/tushare/data/5min/
sz300334 has saved on D:/tushare/data/5min/
sz300335 has saved on D:/tushare/data/5min/
sz300336 has saved on D:/tushare/data/5min/
sz300337 has saved on D:/tushare/data/5min/
sz300338 has saved on D:/tushare/data/5min/
sz300339 has saved on D:/tushare/data/5min/
sz300340 has saved on D:/tushare/data/5min/
sz300341 has saved on D:/tushare/data/5min/
sz300342 has saved on D:/tushare/data/5min/
sz300343 has saved on D:/tushare/data/5min/
sz300344 has saved on D:/tushare/data/5min/
sz300345 has saved on D:/tushare/data/5min/
sz300346 has saved on D:/tushare/data/5min/
sz300347 has saved on D:/tushare/data/5min/
sz300348 has saved on D:/tushare/data/5min/
sz300349 has saved on D:/tushare/data/5min/
sz300350 has saved on D:/tushare/data/5min/
sz300351 has saved on D:/tushare

sz300518 has saved on D:/tushare/data/5min/
sz300519 has saved on D:/tushare/data/5min/
sz300520 has saved on D:/tushare/data/5min/
sz300521 has saved on D:/tushare/data/5min/
sz300522 has saved on D:/tushare/data/5min/
sz300523 has saved on D:/tushare/data/5min/
sz300525 has saved on D:/tushare/data/5min/
sz300526 has saved on D:/tushare/data/5min/
sz300527 has saved on D:/tushare/data/5min/
sz300528 has saved on D:/tushare/data/5min/
sz300529 has saved on D:/tushare/data/5min/
sz300530 has saved on D:/tushare/data/5min/
sz300531 has saved on D:/tushare/data/5min/
sz300532 has saved on D:/tushare/data/5min/
sz300533 has saved on D:/tushare/data/5min/
sz300534 has saved on D:/tushare/data/5min/
sz300535 has saved on D:/tushare/data/5min/
sz300536 has saved on D:/tushare/data/5min/
sz300537 has saved on D:/tushare/data/5min/
sz300538 has saved on D:/tushare/data/5min/
sz300539 has saved on D:/tushare/data/5min/
sz300540 has saved on D:/tushare/data/5min/
sz300541 has saved on D:/tushare

sz300716 has saved on D:/tushare/data/5min/
sz300717 has saved on D:/tushare/data/5min/
sz300718 has saved on D:/tushare/data/5min/
sz300719 has saved on D:/tushare/data/5min/
sz300720 has saved on D:/tushare/data/5min/
sz300721 has saved on D:/tushare/data/5min/
sz300722 has saved on D:/tushare/data/5min/
sz300723 has saved on D:/tushare/data/5min/
sz300724 has saved on D:/tushare/data/5min/
sz300725 has saved on D:/tushare/data/5min/
sz300726 has saved on D:/tushare/data/5min/
sz300727 has saved on D:/tushare/data/5min/
sz300729 has saved on D:/tushare/data/5min/
sz300730 has saved on D:/tushare/data/5min/
sz300731 has saved on D:/tushare/data/5min/
sz300732 has saved on D:/tushare/data/5min/
sz300733 has saved on D:/tushare/data/5min/
sz300735 has saved on D:/tushare/data/5min/
sz300736 has saved on D:/tushare/data/5min/
sz300737 has saved on D:/tushare/data/5min/
sz300738 has saved on D:/tushare/data/5min/
sz300739 has saved on D:/tushare/data/5min/
sz300740 has saved on D:/tushare

In [25]:
def getstockdata(code='000001',year_s=2015,month_s=2,day_s=1,year_e=2019,month_e=1,day_e=25,file_address='D:\\tushare\\data'):
    """
    获取单个股票处理后的dataframe格式数据,code 默认为000001，默认start='2016-01-01',end='2019-1-25'
    """
    dt1=datetime.date(year_s,month_s,day_s);
    dt2=datetime.date(year_e,month_e,day_e);
    data=pd.read_csv(file_address+'\\'+ code, sep=',',encoding='gb2312')
    #data.set_index('date',inplace=True)
    data.index = pd.to_datetime(data['date'])
    data.drop(columns=['Unnamed: 0','date'],axis=1,inplace=True)

    return data.ix[dt1:dt2]